In [1]:
from vplants.autowig import autowig

In [2]:
from path import path

rootdir = path('../../statiskit/').abspath()

includedir = rootdir + "/core/build-scons/include"
headers = [str(f) for f in includedir.walkfiles('*.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I/usr/include', '-I' + str(includedir), '-D__STDC_CONSTANT_MACROS',
         '-I/usr/local/lib/clang/3.7.0/include', '-D__STDC_LIMIT_MACROS']

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=3, cache=rootdir+'/.AutoWIG')#, force=True)

In [3]:
rends = asg.functions('::std::vector<std::basic_string<char>, std::allocator<std::basic_string<char> > >::rend')
rends[0].boost_python_export, rends[1].boost_python_export

(True, True)

In [4]:
autowig.middle_end.plugin = 'default'
autowig.middle_end(asg)

primaries = [header for header in asg.files(header=True) if header.is_primary]
for hdr in asg.files(header=True):
    if not hdr.is_primary and hdr.include in primaries:
        hdr.is_primary = True
        
for inc in asg.directories():
    if inc.as_include:
        if inc.globalname + 'c++/' in asg:
            for vsn in asg[inc.globalname + 'c++/'].directories:
                vsn.as_include = True
                for hdr in vsn.files:
                    if hdr.suffix == '':
                        hdr.is_primary = True
        elif inc.globalname + 'boost/' in asg:
            for hdr in sg[inc.globalname + 'boost/'].files:
                hdr.is_primary = True
                
for tpl in asg.classes('.*(unique|weak|shared)_ptr$', templated=True,
                       specialized=False):
    tpl.is_smart_pointer = True

In [5]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

In [6]:
if 'struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >':
    asg['struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >'].boost_python_export = False
if 'class ::arma::SpMat<double>' in asg:
    for mtd in asg['class ::arma::SpMat<double>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<double>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<double>' in asg:
    for mtd in asg['class ::arma::SpSubview<double>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False

In [7]:
rends[0].boost_python_export, rends[1].boost_python_export

(True, True)

In [8]:
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/core/src/wrapper',
                 pattern = '.*statiskit.*',
                 prefix = '_')
print rends[0].boost_python_export, rends[1].boost_python_export
autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)
print rends[0].boost_python_export, rends[1].boost_python_export
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/misc/src/wrapper',
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/wrapper/__core.cpp',
                 package = 'statiskit.core')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/wrapper/__misc.cpp',
                 package = 'statiskit.misc')
autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/statiskit/core/_core.py',
                 module = rootdir + '/core/src/wrapper/__core.cpp')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/statiskit/misc/_misc.py',
                 module = rootdir + '/misc/src/wrapper/__misc.cpp')

True True
True True


In [9]:
asg['class ::arma::SpSubview<double>'].boost_python_export

/home/pfernique/Desktop/Git/statiskit/misc/src/wrapper/_arma_sp_subview_35365bb0e9a15ad1a7bbb531af8db059.cpp

In [10]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/(core|misc)/src/(wrapper|statiskit)/.*')

In [11]:
autowig.scons(rootdir + '/misc', '-j3')

In [12]:
autowig.scons(rootdir + '/core', '-j3')

In [13]:
mtds = asg['class ::std::vector<double, std::allocator<double> >'].methods('.*begin.*')
mtds

[::std::vector<double, std::allocator<double> >::begin::8586b02d-882f-5625-8704-49546ef111b6,
 ::std::vector<double, std::allocator<double> >::begin::0aa5fdfc-c294-5df3-8c85-7dcad5566e45,
 ::std::vector<double, std::allocator<double> >::rbegin::7af8af85-3d23-58e4-887d-7fd6350be1da,
 ::std::vector<double, std::allocator<double> >::rbegin::28f8735c-bfc9-5241-9c6d-2ce0dfcb9717,
 ::std::vector<double, std::allocator<double> >::cbegin::f81eb128-bd51-5553-ad5d-bf383e387820,
 ::std::vector<double, std::allocator<double> >::crbegin::1380f30b-2eb3-5603-a479-a901ec8e8d9f]

In [14]:
print mtds[0].result_type

class ::__gnu_cxx::__normal_iterator<double *, std::vector<double, std::allocator<double> > >


In [18]:
spcs = asg['class ::std::shared_ptr'].specializations()

In [19]:
[spc.boost_python_export for spc in spcs]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]